In [ ]:
#%%
import numpy as np
import copy 
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_probability as tfp
# Main C3 objects
from c3.c3objs import Quantity as Qty
from c3.parametermap import ParameterMap as PMap
from c3.experiment import Experiment as Exp
from c3.model import Model as Mdl
from c3.generator.generator import Generator as Gnr

# Building blocks
import c3.generator.devices as devices
import c3.signal.gates as gates
import c3.libraries.chip as chip
import c3.signal.pulse as pulse
import c3.libraries.tasks as tasks

# Libs and helpers
import c3.libraries.algorithms as algorithms
import c3.libraries.hamiltonians as hamiltonians
import c3.libraries.fidelities as fidelities
import c3.libraries.envelopes as envelopes
import c3.utils.qt_utils as qt_utils
import c3.utils.tf_utils as tf_utils

#%matplotlib widget



In [ ]:
qubit_levels = 2
freq_q1 = 4e9
anharm_q1 = -210e6
t1_q1 = 27e-6
t2star_q1 = 39e-6
qubit_temp = 50e-3
phi_1 = 0.1
phi_0_1 = 10.0
d_1 = 0.0


q1 = chip.Transmon(
        name = "Q1",
        desc = "Qubit  1",
        freq = Qty(
                value = freq_q1,
                min_val = 3.995e9,
                max_val = 4.005e9,
                unit = "Hz 2pi"
        ),
        hilbert_dim = qubit_levels,
        t1 = Qty(
                value = t1_q1,
                min_val = 1e-6,
                max_val = 90e-6,
                unit = "s"
        ),
        t2star=Qty(
                value=t2star_q1,
                min_val=10e-6,
                max_val=90e-6,
                unit="s"
        ),
        temp=Qty(
                value=qubit_temp,
                min_val=0.0,
                max_val=0.12,
                unit="K"
        ),
        phi=Qty(
                value=phi_1,
                max_val=5.0,
                min_val=0.0,
                unit = "Wb"
        ),
        phi_0 = Qty(
            value = phi_0_1,
            max_val = 11.0,
            min_val = 9.0,
            unit = "Wb"
        ),
        d = Qty(
            value=d_1,
            max_val=0.1,
            min_val=-0.1,
            unit = ""
        )
)

# %%

qubit_levels = 2
freq_q2 = 4.5e9
anharm_q2 = -240e6
t1_q2 = 23e-6
t2star_q2 = 31e-6
qubit_temp = 50e-3
phi_2 = 0.1  
phi_0_2 = 10.0
d_2 = 0.0

q2 = chip.Transmon(
        name = "Q2",
        desc = "Qubit  2",
        freq = Qty(
                value = freq_q2,
                min_val = 4.495e9,
                max_val = 4.505e9,
                unit = "Hz 2pi"
        ),
        hilbert_dim = qubit_levels,
        t1 = Qty(
                value = t1_q2,
                min_val = 1e-6,
                max_val = 90e-6,
                unit = "s"
        ),
        t2star=Qty(
                value=t2star_q2,
                min_val=10e-6,
                max_val=90e-6,
                unit="s"
        ),
        temp=Qty(
                value=qubit_temp,
                min_val=0.0,
                max_val=0.12,
                unit="K"
        ),
        phi=Qty(
                value=phi_2,
                max_val=5.0,
                min_val=0.0,
                unit = "Wb"
        ),
        phi_0 = Qty(
            value = phi_0_2,
            max_val = 11.0,
            min_val = 9.0,
            unit = "Wb"
        ),
        d = Qty(
            value=d_2,
            max_val=0.1,
            min_val=-0.1,
            unit = ""
        )        
)

# %%

qubit_levels = 2
freq_coupler = 6.5e9
anharm_coupler = -200e6
t1_coupler = 20e-6
t2star_coupler = 30e-6
qubit_temp = 50e-3
phi_coupler = 0.1  
phi_0_coupler = 10.0
d_c = 0.0

c1 = chip.Transmon(
        name = "C1",
        desc = "Tunable coupler coupling Qubit 1 and Qubit 2",
        freq = Qty(
                value = freq_coupler,
                min_val = 6.495e9,
                max_val = 6.505e9,
                unit = "Hz 2pi"
        ),
        hilbert_dim = qubit_levels,
        t1 = Qty(
                value = t1_coupler,
                min_val = 1e-6,
                max_val = 90e-6,
                unit = "s"
        ),
        t2star=Qty(
                value=t2star_coupler,
                min_val=10e-6,
                max_val=90e-6,
                unit="s"
        ),
        temp=Qty(
                value=qubit_temp,
                min_val=0.0,
                max_val=0.12,
                unit="K"
        ),
        phi=Qty(
                value=phi_coupler,
                max_val=5.0,
                min_val=0.0,
                unit = "Wb"
        ),
        phi_0 = Qty(
            value = phi_0_coupler,
            max_val = 11.0,
            min_val = 9.0,
            unit = "Wb"
        ),
        d = Qty(
            value=d_c,
            max_val=0.1,
            min_val=-0.1,
            unit = ""
        )
)

# %%

coupling_strength_1 = 20e6
coupling_strength_2 = 25e6
q1c1 = chip.Coupling(
        name="Q1-C1",
        desc="Coupling",
        comment="Coupling between qubit 1 and Coupler",
        connected=["Q1","C1"],
        strength=Qty(
                value=coupling_strength_1,
                min_val=-1*1e3,
                max_val=200e6,
                unit="Hz 2pi"
        ),
        hamiltonian_func=hamiltonians.int_XX
)

c1q2 = chip.Coupling(
        name="C1-Q2",
        desc="Coupling",
        comment="Coupling between qubit 2 and Coupler",
        connected=["C1","Q2"],
        strength=Qty(
                value=coupling_strength_2,
                min_val=-1*1e3,
                max_val=200e6,
                unit="Hz 2pi"
        ),
        hamiltonian_func=hamiltonians.int_XX
)

# %%


drive1 = chip.Drive(
        name="d1",
        desc="Drive 1",
        comment = "Drive line 1 on qubit 1",
        connected = ["Q1"],
        hamiltonian_func = hamiltonians.x_drive
)

drive2 = chip.Drive(
        name="d2",
        desc="Drive 2",
        comment = "Drive line 2 on qubit 2",
        connected = ["Q2"],
        hamiltonian_func = hamiltonians.x_drive
)
""""
drive_coupler = chip.Drive(
        name = "dc",
        desc = "Drive 1",
        comment = "Drive line on the coupler",
        connected = ["C1"],
        hamiltonian_func = hamiltonians.x_drive
)
"""

# %%

m00_q1 = 0.97
m01_q1 = 0.04
m00_q2 = 0.96
m01_q2 = 0.05
one_zeros = np.array([0] * qubit_levels)
zero_ones = np.array([1] * qubit_levels)
one_zeros[0] = 1
zero_ones[0] = 0

val1 = one_zeros * m00_q1 + zero_ones * m01_q1
val2 = one_zeros * m00_q2 + zero_ones * m01_q2

min_val = one_zeros * 0.8 + zero_ones * 0.0
max_val = one_zeros * 1.0 + zero_ones * 0.2

confusion_row1 = Qty(value=val1, min_val=min_val, max_val=max_val, unit="")
confusion_row2 = Qty(value=val2, min_val=min_val, max_val=max_val, unit="")

conf_matrix = tasks.ConfusionMatrix(Q1=confusion_row1, Q2=confusion_row2)


# %%

init_temp = 50e-3
init_ground = tasks.InitialiseGround(
        init_temp = Qty(
                value = init_temp,
                min_val = -0.001,
                max_val = 0.22,
                unit = "K"
        )
)

# %%

model = Mdl(
        [q1,c1,q2],
        [drive1, drive2, q1c1, c1q2],
        [conf_matrix, init_ground]
)
model.set_lindbladian(False)
model.set_dressed(True)